In [2]:
# easyocr 모델 설치
# !pip install easyocr

# py-hanspell 라이브러리 설치
# !pip install git+https://github.com/ssut/py-hanspell.git
import easyocr
import cv2
import math
from hanspell import spell_checker
import pandas as pd


## 1) 비디오 타임스탬프 및 텍스트 추출
class Bindio:
        
    def video_to_frame(video_path) :


        frame_images=[]
        vidcap = cv2.VideoCapture(video_path)
        fps = vidcap.get(cv2.CAP_PROP_FPS)
        timestamps = [vidcap.get(cv2.CAP_PROP_POS_MSEC)]
        calc_timestamps = [0.0]

        count=0
        success=True

        while(vidcap.isOpened()):
            vidcap.set(cv2.CAP_PROP_POS_MSEC, (count*1000))
            success, image = vidcap.retrieve()

            # print('{}.sec reading a new frame: {}'.format(count, success))

            frame_exists, curr_frame = vidcap.read()
            if frame_exists:
                timestamps.append(vidcap.get(cv2.CAP_PROP_POS_MSEC))
                calc_timestamps.append(calc_timestamps[-1] + 1000/fps)
            else:
                break

            frame_images.append(image)

            # 프레임 추출을 위한 초 간격 설정
            count += 3

        vidcap.release()

        # 타임스탬프 생성
        times = []

        for t in timestamps :
            sec = t/1000
            times.append('{}:{:02d}'.format(math.trunc(sec/60), math.ceil(sec%60)))

        # 프레임내의 텍스트 추출
        reader = easyocr.Reader(['ko','en'], gpu=True)
        result = {}
        for i in range(len(frame_images)-1):
            result[times[i]] = [reader.readtext(frame_images[i], detail = 0)]

        return result


    ## 2) 맞춤법 검사
    def spell_check(result) :

        list = []

        for index, (key, val) in enumerate(result.items()) :
            # 딕셔너리 내 리스트 문자를 합치기(합칠 때 공백을 구분자로 사용)
            cont = ' '.join(result[key][0])

            # 스펠 체크
            spell_check = spell_checker.check(cont)

            # 결과 dict로 변환
            dict_result = spell_check.as_dict()
            list.append(
                {'timestamps' : key,
                'original' : dict_result['original'], 
                'checked' : dict_result['checked']}
            )

        # 데이터프레임화
        df = pd.DataFrame(list, columns = ['timestamps', 'original', 'checked'])

        return df


    ## 3) 중복 제거
    def split_duplicate(df) :

        row_index = []

        for i in range(len(df)-1) :
            list1 = df['original'][i].split()
            list2 = df['original'][i+1].split()

            intersection = set(list1) & set(list2)  

            try :
                score = round(len(intersection) / len(list1), 2)

                # 스코어 조정을 통해 중복 제거 미세 조정
                if score >= 0.5 :
                    row_index.append(i+1)
                else :
                    pass

                df_dup = df.drop(index = row_index, axis = 0)
            except :
                pass

        # 데이터 최종 정리
        indexs = df_dup[df_dup['original'] == ''].index
        df_dup.drop(indexs, inplace = True)

        return df_dup
  
  
## 사용 코드
# video_to_frame('video Path')
# spell_chek(result)
# split_duplicate(df)

In [ ]:
Bindio.video_to_frame()